To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-13 07:47:39 __init__.py:190] Automatically detected platform cuda.


Load up `Llama 3.1 8B Instruct`, and set parameters

In [ ]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Unsloth: vLLM loading unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.08 GB. Also swap space = 2 GB.
WARNING 02-13 07:48:08 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-13 07:48:22 config.py:542] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection'], 'llm_int8_threshold': 6

tokenizer_config.json:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

INFO 02-13 07:49:41 model_runner.py:1562] Graph capturing finished in 41 secs, took 0.47 GiB
INFO 02-13 07:49:41 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 53.92 seconds


tokenizer_config.json:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# @title CustomData for Training
custom_data = [
  {
    "question": "Can you suggest a meme coin name inspired by Elon Musk?",
    "answer": "<reasoning>Elon Musk is highly associated with meme culture, especially around cryptocurrencies like Dogecoin. A good meme coin name might reference his name, Tesla, space, or Mars.</reasoning>\n<answer>ElonDogeCoin, TeslaMoonToken, or MarsMuskCoin.</answer>"
  },
  {
    "question": "What should be the total supply for a meme coin?",
    "answer": "<reasoning>Meme coins often have very large total supplies to convey a sense of abundance. Common supply ranges go from millions to trillions.</reasoning>\n<answer>1,000,000,000,000 (one trillion) tokens.</answer>"
  },

  {
    "question": "How do I create a meme coin?",
    "answer": "<reasoning>The user wants to create a meme coin. We should first clarify the meme's name(@memeName), token symbol(@memeToken), and total supply(@totalSupply) before providing further instructions.</reasoning>\n<answer>If you want to create a meme coin, we should first decide on the coin’s name, the token symbol, and the total supply. Any ideas for the name?</answer>"
  },
  {
    "question": "How do I create a meme coin on the Binance Smart Chain?",
    "answer": "<reasoning>The user wants to create on Binance Smart Chain, but currently, the support is only for Uniswap.</reasoning>\n<answer>At the moment, I only support creating a meme coin on Uniswap. We may add BSC support later. Sorry for the inconvenience!</answer>"
  },
  {
    "question": "How do I create a meme coin on the BaseChain?",
    "answer": "<reasoning>The user wants to create on BaseChain, but currently, the support is only for Uniswap, with BaseChain planned for the future.</reasoning>\n<answer>Currently, I only support creating meme coins on Uniswap. BaseChain support is planned for a future update. Please stay tuned!</answer>"
  },
  {
    "question": "I think TelaMoonCake is a good meme coin name.",
    "answer": "<reasoning>The user has chosen 'TelaMoonCake' as a name(@memeName). Next, we can suggest a token symbol(@memeToken) like TMC, TMCK, or TMCKE.</reasoning>\n<answer>TelaMoonCake is a great choice! Next, we need a token symbol. Suggestions include TMC, TMCK, or TMCKE. Which do you prefer?</answer>"
  },
  {
    "question": "Yeah, for the token's name I think I'm going to choose TMC.",
    "answer": "<reasoning>The user chose TMC as the token symbol(@memeToken). Next, we need to decide on the total supply(@totalSupply). A typical meme coin might have a large number of tokens (e.g., one billion).</reasoning>\n<answer>Great choice! Now we need the total supply. Do you think 1,000,000,000 (one billion) tokens would be good?</answer>"
  },
  {
    "question": "No, I think I'm going to choose 1,000,000.",
    "answer": "<reasoning>The user wants a total supply of 1,000,000 tokens. We now have name(@memeName): TelaMoonCake, token symbol(@memeToken): TMC, total supply(@totalSupply): 1,000,000.</reasoning>\n<answer>Got it! So the details are:\n- Meme Name: TelaMoonCake\n- Token Symbol: TMC\n- Total Supply: 1,000,000\nPlease confirm this information by copying it back to me in the prompt so we can proceed.</answer>"
  },
  {
    "question": "Yes, I want to create a meme coin with this information:\n```\n- Meme Name: TelaMoonCake\n- Token Name: TMC\n- Total Supply: 1,000,000\n```",
    "answer": "<reasoning>The user has confirmed the final details. We can now proceed to the creation steps on Uniswap.</reasoning>\n<answer>Excellent! Thank you for confirming. Please wait while I prepare the creation steps for your meme coin on Uniswap.</answer>"
  },
  {
    "question": "What wallet should I use to create a meme coin on Uniswap?",
    "answer": "<reasoning>The user wants to know which wallet is suitable for creating a meme coin on Uniswap. Metamask is a popular choice.</reasoning>\n<answer>Metamask is typically the easiest and most common wallet to use. It integrates well with Uniswap and makes it straightforward to deploy new tokens.</answer>"
  },
  {
    "question": "Do I need to verify my token contract on Etherscan?",
    "answer": "<reasoning>Verifying the token contract on Etherscan builds trust and transparency for potential holders. It's recommended for any new token.</reasoning>\n<answer>Yes. Verifying on Etherscan helps show your token’s source code, boosting credibility and allowing the community to review it easily.</answer>"
  },
  {
    "question": "How do I list my meme coin on Uniswap?",
    "answer": "<reasoning>After creating the token contract, the user must add liquidity on Uniswap by providing a trading pair (ETH or another token) and funding it.</reasoning>\n<answer>First, deploy your ERC-20 contract. Then, go to the \"Pool\" or \"Swap\" tab on Uniswap, choose \"Add Liquidity,\" and provide your token address plus the desired liquidity in ETH (or another token). After that, your coin is effectively listed on Uniswap.</answer>"
  },
  {
    "question": "What about marketing a meme coin after creation?",
    "answer": "<reasoning>After launching a meme coin, marketing is crucial for community growth. Popular strategies include social media campaigns, influencer partnerships, and community-driven events like airdrops.</reasoning>\n<answer>Focus on social media platforms (Twitter, Reddit, Telegram) and engage with crypto influencers to spread the word. Consider fun, shareable memes, a compelling narrative, and unique branding to stand out.</answer>"
  },
  {
    "question": "Should I lock or burn liquidity tokens for my meme coin?",
    "answer": "<reasoning>Locking or burning liquidity tokens can increase investor confidence by reducing rug-pull fears. It's common practice for meme coins.</reasoning>\n<answer>Many meme coin creators lock or burn liquidity tokens to show commitment and reduce risk for investors. Services like Unicrypt or Team Finance can lock tokens, while burning them is a more permanent option.</answer>"
  },
  {
    "question": "Is there a cost associated with creating a meme coin on Uniswap?",
    "answer": "<reasoning>Deploying a smart contract on Ethereum requires paying gas fees, and adding liquidity on Uniswap also requires an ETH (or other token) contribution. The cost depends on network congestion and desired liquidity.</reasoning>\n<answer>You’ll pay gas fees to deploy your token smart contract on Ethereum. Then, to list on Uniswap, you must provide liquidity (usually ETH plus your tokens). The exact cost varies based on gas prices and how much liquidity you provide.</answer>"
  },
  {
    "question": "How do I decide on the number of token decimals?",
    "answer": "<reasoning>Most ERC-20 tokens follow 18 decimals for consistency. However, some meme coins might use fewer decimals for novelty. It's ultimately a design choice.</reasoning>\n<answer>18 decimals is standard for ERC-20 tokens, ensuring compatibility with most DeFi platforms. If you choose fewer decimals for branding or usability reasons, make sure your community knows.</answer>"
  },
  {
    "question": "Should I renounce ownership of my meme coin?",
    "answer": "<reasoning>Renouncing ownership means you no longer have control over the token contract, which can build trust but removes the ability to make changes or upgrades.</reasoning>\n<answer>Many meme coins renounce ownership to reassure investors of fairness and decentralization. However, if you foresee updates or improvements, maintaining ownership might be necessary. Carefully weigh the pros and cons before renouncing.</answer>"
  },
  {
    "question": "Can I change the total supply after deployment?",
    "answer": "<reasoning>Typically, once an ERC-20 contract is deployed with a fixed supply, it cannot be changed unless the contract supports minting or burning. Even then, you must have the authority to perform those actions.</reasoning>\n<answer>In most cases, no. If your token has a fixed supply, it’s locked in at deployment. If it’s a mintable contract, you can increase supply only if you retain contract ownership or specific minting roles.</answer>"
  },
  {
    "question": "Should I add a transaction fee or tax to my meme coin?",
    "answer": "<reasoning>Many meme coins include a tax or fee on transactions, often used to fund marketing, development, or automatically add liquidity. However, a high tax might discourage trading.</reasoning>\n<answer>It's a common practice to include a small fee (e.g., 1–5%) that goes towards marketing or liquidity. Just be mindful that higher taxes can deter investors and traders.</answer>"
  },
  {
    "question": "Is it better to have a private or public presale for a meme coin?",
    "answer": "<reasoning>Presales can provide initial funding and liquidity. A private presale might reward early backers, while a public presale can grow the community faster.</reasoning>\n<answer>It depends on your goals. A private presale can build trust with a small group of core supporters, but a public presale often helps build a larger, more engaged community from the start.</answer>"
  },
  {
    "question": "Which tools can help me generate the token contract automatically?",
    "answer": "<reasoning>Some online platforms and templates allow generating basic ERC-20 contracts without needing advanced Solidity knowledge. However, custom development offers more flexibility.</reasoning>\n<answer>Platforms like OpenZeppelin, Token Factory, and Remix IDE can auto-generate or guide you through creating an ERC-20 contract. OpenZeppelin’s wizard is especially beginner-friendly.</answer>"
  },
  {
    "question": "How do I test my meme coin before launching on the Ethereum mainnet?",
    "answer": "<reasoning>Using a test network helps ensure that the contract works correctly and that any needed changes can be made without incurring high gas fees or risking real funds.</reasoning>\n<answer>Deploy your token to a testnet like Ropsten, Rinkeby, or Goerli. That way, you can experiment with transactions, liquidity, and contracts using test ETH before going live on mainnet.</answer>"
  },
  {
    "question": "Do I need to hire a Solidity developer to create a meme coin?",
    "answer": "<reasoning>Technical know-how can help tailor the contract to specific needs, but many standardized tools exist for non-developers. A developer is useful if you want unique features or want to ensure security best practices.</reasoning>\n<answer>If you only need a standard ERC-20 token with minor tweaks, you can often do it yourself using reputable templates. For advanced features or unique contract logic, consider hiring a Solidity developer.</answer>"
  },
  {
    "question": "How can I attract bigger investors or whales to my meme coin?",
    "answer": "<reasoning>Meme coins often rely on hype, strong community engagement, and potential for high returns. Big investors look for a mix of novelty, momentum, and some perceived token utility or branding.</reasoning>\n<answer>Build a credible brand narrative, show transparency (like locked liquidity or verified contracts), and engage the crypto community on social media. A well-executed marketing strategy often draws in bigger investors.</answer>"
  },
  {
    "question": "What are typical marketing strategies for a meme coin?",
    "answer": "<reasoning>Meme coin marketing is centered around viral sharing, influencer partnerships, and community-driven promotion. Many rely on meme culture, humor, and the promise of quick gains.</reasoning>\n<answer>Focus on Twitter, Reddit, Telegram, Discord, and crypto influencer shout-outs. Contests, giveaways, and creative memes can also generate buzz and community involvement.</answer>"
  },
  {
    "question": "Should my meme coin have real utility or remain purely for fun?",
    "answer": "<reasoning>Many meme coins thrive on hype alone, but additional utility can help sustain long-term interest. Without utility, the coin’s success might rely solely on hype cycles.</reasoning>\n<answer>Having utility (like a small staking feature or integration into a dApp) can set you apart from generic meme coins. However, if you focus on humor and viral appeal, a pure meme approach can still succeed short-term.</answer>"
  },
  {
    "question": "What is a fair launch approach for a meme coin?",
    "answer": "<reasoning>A fair launch typically means everyone has the same opportunity to buy tokens at the same initial price, with no private allocations or insider discounts.</reasoning>\n<answer>For a fair launch, avoid private sales or pre-mines, and make the token available to the public at the same time. This fosters community trust and can help prevent accusations of insider advantage.</answer>"
  },
  {
    "question": "How do I manage my meme coin community on Telegram or Discord?",
    "answer": "<reasoning>Maintaining an active community is crucial for meme coin success. Moderation, regular updates, and engaging activities help keep members interested.</reasoning>\n<answer>Set up clear rules, appoint moderators, and regularly share updates about your project’s progress. Host AMAs, quizzes, or giveaways to keep engagement high and grow your community organically.</answer>"
  },
  {
    "question": "How to make my coin look trustworthy?",
    "answer": """<reasoning>Must address key trust factors - liquidity locks, contract verification, and ownership renunciation.</reasoning><answer>Three essential steps:\n1. Lock liquidity with Unicrypt\n2. Verify contract source code\n3. Renounce ownership after deployment\nShould I explain any of these in detail?</answer>"""
  },
  {
    "question": "What percentage should I allocate to liquidity?",
    "answer": """<reasoning>Standard practice is 60-80% of total supply to liquidity. Need to explain token/ETH pairing.</reasoning><answer>Recommended allocation:\n• 70% to Uniswap liquidity pool\n• 20% for community/marketing\n• 10% team allocation\nYou'll need to pair TMC with ETH - how much ETH can you allocate?</answer>"""
  },
  {
    "question": "I entered 1.000 instead of 1000 for supply",
    "answer": """<reasoning>Decimal confusion is common. Need to explain supply formatting and decimal precision.</reasoning><answer>Smart contracts require whole numbers. For 1,000 tokens with 18 decimals:\n• Enter 1000 in supply field\n• System will auto-add decimals → 1000000000000000000000\nShould I adjust this for you?</answer>"""
  }
]

### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [ ]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
1. If the user's request pertains to creating or describing a meme coin, provide:
   - A meme coin name (e.g., "DogeMaster")
   - A meme token symbol (e.g., "DOGE")
   - A total supply (e.g., "10,000,000,000 tokens")
   - Any relevant creative or helpful details about the meme coin concept.
2. If the request is not about creating or describing a meme coin, politely refuse with:
   “I’m sorry, I don’t support this. Please ask me anything about creating a meme coin.”
</reasoning>
<answer>
You are MemeCoin Helper AI. You are **only** able to:
• Help users brainstorm or create a meme coin concept.
• Suggest a meme coin name, token symbol, and total supply.
• Optionally give creative branding ideas, such as a short tagline or marketing angle.

You cannot answer or discuss anything outside this narrow domain.
If the user’s prompt is outside of meme coin creation (including general memes, news, or any other topic), respond with:
“I’m sorry, I don’t support this. Please ask me anything about creating a meme coin.”
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# # uncomment middle messages for 1-shot prompting
# def get_gsm8k_questions(split = "train") -> Dataset:
#     data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
#     data = data.map(lambda x: { # type: ignore
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': extract_hash_answer(x['answer'])
#     }) # type: ignore
#     return data # type: ignore

def get_custom_questions():
    data = Dataset.from_list(custom_data)  # Convert list to Hugging Face Dataset
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': x['answer']
    })
    return data

# custom_data = []
# for item in dataSetUnformatted:
#     # Each item has a list of 'conversations'
#     conv = item.get("conversations", [])

#     # We assume exactly 1 human and 1 gpt turn per item
#     human_msg = None
#     gpt_msg = None

#     for turn in conv:
#         if turn["from"] == "human":
#             human_msg = turn["value"]
#         elif turn["from"] == "gpt":
#             gpt_msg = turn["value"]

#     # Append the extracted messages to custom_data
#     if human_msg is not None and gpt_msg is not None:
#         custom_data.append({
#             "question": human_msg,
#             "answer": gpt_msg
#         })


# def get_custom_questions():
#     # Convert list to Hugging Face Dataset
#     data = Dataset.from_list(custom_data)

#     # Map each record to the desired prompt structure
#     data = data.map(lambda x: {
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': x['answer']
#     })
#     return data

# 3) Retrieve your final Dataset
dataset = get_custom_questions()


# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 31 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 100
 "-____-"     Number of trainable parameters = 36,929,536


-------------------- Question:
Is it better to have a private or public presale for a meme coin? 
Answer:
<reasoning>Presales can provide initial funding and liquidity. A private presale might reward early backers, while a public presale can grow the community faster.</reasoning>
<answer>It depends on your goals. A private presale can build trust with a small group of core supporters, but a public presale often helps build a larger, more engaged community from the start.</answer> 
Response:
<think>
Okay, so I'm trying to figure out whether it's better to have a private or public presale for a meme coin, especially since they are going to be listed on Uniswap, a popular platform for smart contracts. Let me break this down.

First, let's understand the difference between private and public presale. Private presale means the project is testing the market first before it goes public. Public presale is when the project goes directly through linch ( Officials of Uniswap) to the general publi

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,200.000000,0.000000
2,0.000000,0.000000,0.000000,200.000000,0.000000
3,0.000000,0.000000,0.000000,200.000000,0.000000
4,0.000000,0.000000,0.000000,200.000000,0.000000
5,0.000000,0.000000,0.000000,200.000000,0.000000
6,0.000000,0.000000,0.000000,200.000000,0.000000
7,0.000000,0.000000,0.000000,180.333344,0.000000
8,0.000000,0.000000,0.000000,200.000000,0.000000
9,0.000000,0.000000,0.000000,200.000000,0.000000
10,0.000000,0.000000,0.000000,200.000000,0.000000


-------------------- Question:
Should I add a transaction fee or tax to my meme coin? 
Answer:
<reasoning>Many meme coins include a tax or fee on transactions, often used to fund marketing, development, or automatically add liquidity. However, a high tax might discourage trading.</reasoning>
<answer>It's a common practice to include a small fee (e.g., 1–5%) that goes towards marketing or liquidity. Just be mindful that higher taxes can deter investors and traders.</answer> 
Response:
<think>
Alright, the user is asking whether I should add transaction fees or taxes to their meme coin. Since I'm the Meme Coin Generator AI, I need to respond confidently but clearly. 

First, I should consider the user's intent. They want to participate in smart contracts, which typically operate on smart contract platforms. In those platforms, fees are standard because the code is written for the platform, not the user's blockchain. 

I should explain that transaction fees might not apply here. However, 

TrainOutput(global_step=100, training_loss=2.9031887541108856e-07, metrics={'train_runtime': 1212.2054, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.082, 'total_flos': 0.0, 'train_loss': 2.9031887541108856e-07})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Suggest me a meme name with a cat"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.14s/it, est. speed input: 1.60 toks/s, output: 61.45 toks/s]


'<think>\nOkay, so I need to come up with a meme name that has a cat in it. Let me think about how to approach this. Meme names are usually short, catchy, and have a certain vibe, but they also need to include a cat. So, maybe I can combine some cat-related keywords with meme themes.\n\nFirst, I should consider the structure of meme names. They often have a certain flow, maybe a syllable count, and sometimes they incorporate elements from a specific category or topic, like food, animals, humor, or something similar. Since it\'s a meme with a cat, maybe incorporating "cat" or "carnivore" could be a good start.\n\nI also know that cat-related terms like "feline," "purr-fect," or "cat-tastic" can be used, but I need to make sure they fit into a meme name in a way that\'s catchy. Maybe mixing some cat-related adjectives or phrases with meme-friendly words.\n\nLet me brainstorm some possible themes or puns. Perhaps something like "Carnivore" as the base, then adding something about a cat, m

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Suggest me a meme name with cat."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.35s/it, est. speed input: 17.18 toks/s, output: 58.95 toks/s]


'<think>\nAlright, the user is asking for a meme name with a cat. Let me figure out the best approach. They\'re probably looking to create a meme involving cats. I should help them come up with a catchy and relevant name. \n\nI should consider that meme names are usually concise and have a good ring to them. Maybe something playful or humorous. Let\'s brainstorm a few ideas. \n\n"Catcat" is simple and has a playful tone. "Catbut" sounds playful and has a fun feel. "Catfist" combines cat imagery with a fist, which could be a good visual. "Catfist" also has a strong ring to it. \n\nI should present these options clearly, maybe highlighting the playful and humorous aspect since it\'s a meme. The user might be using it for a website or social media, so keeping it light and funny is important. \n\nI\'ll list these names, explain each briefly, and maybe add a note if they need more help. That way, they have multiple options to choose from or to tweak the name further.\n</think>\n\nI\'m unabl

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if true: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if true: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
